<a href="https://colab.research.google.com/github/rocketwolf98/ML_experiments/blob/main/TextGen_Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
filepath = '/content/drive/MyDrive/TextGen/ceb_bible.txt'
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

In [4]:
text = text[300000:800000]

In [31]:
print(text)

g sa tanan nga imo nga anaa sa kaumahan; kay ang tagsatagsa ka tawo kun mananap nga makaplagan diha sa uma, ug dili mahipus sa balay ang nga ulan-nga-yelo magabundak sa ibabaw niya, ug mamatay siya.
02o	9	20	siya nga nahadlok sa pulong ni jehova sa kinataliwad-an sa mga sulogoon ni faraon, nagpakalagiw sa iyang mga binatonan ug sa iyang kahayupan ngadto sa mga balay:
02o	9	21	ug siya nga wala magtagad sa pulong ni jehova, nagpasagad sa iyang mga binatonan ug sa iyang kahayupan sa uma.
02o	9	22	&para; ug si jehova miingon kang moises: tuy-ora ang imong kamot ngadto sa langit aron moabut ang ulan-nga-yelo sa tibook nga yuta sa egipto, sa ibabaw sa mga tawo, ug sa ibabaw sa kamananapan, ug sa ibabaw sa tanan nga balili, sa tibook nga yuta sa egipto.
02o	9	23	ug gituy-od ni moises ang iyang sungkod ngadto sa langit, ug gipadalogdog ni jehova, ug gipaulan ug ulan-nga-yelo ug ang kalayo midalagan ngadto sa yuta: ug gipaulan ni jehova ug ulan-nga-yelo sa ibabaw sa yuta sa egipto.
02o	9	24	bus

In [16]:
#cleaning the text
import re
def clean_text(text):
  tokens = text.split()
  clean_tokens = [token for token in tokens if not re.search(r'\d', token)]

  cleaned_text =' '.join(clean_tokens)
  return cleaned_text

clean_text = clean_text(text)
'''
#clean input sentences
cleaned_sentence = [s.replace('\r','') for s in clean_text]

#rebuilt vocabulary
characters = sorted(list(set(''.join(cleaned_sentence))))
'''

"\n#clean input sentences\ncleaned_sentence = [s.replace('\r','') for s in clean_text]\n\n#rebuilt vocabulary\ncharacters = sorted(list(set(''.join(cleaned_sentence))))\n"

In [19]:
characters = sorted(set(clean_text))

In [20]:
char_to_index = dict((c,i) for i, c in enumerate(characters))
index_to_char = dict((i,c) for i, c in enumerate(characters))

In [21]:
SEQ_LENGTH = 48
STEP_SIZE = 4

In [22]:
sentences = []
next_characters = []

In [23]:
for i in range(0, len(text)- SEQ_LENGTH, STEP_SIZE):
  sentences.append(text[i: i+SEQ_LENGTH])
  next_characters.append(text[i: i+SEQ_LENGTH])

In [25]:
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=bool)
y = np.zeros((len(sentences), len(characters)), dtype=bool)

for i, sentence in enumerate(sentences):
  for t, character in enumerate(sentence):
    if character in char_to_index:
      x[i, t, char_to_index[character]] = 1

  if t < len(sentence) - 1:
    next_char = sentence[t+1]
    if next_char in char_to_index:
      y[i,char_to_index[next_char]] = 1

'''
for i, sentence in enumerate(sentences):
  for t, character in enumerate(sentence):
    x[i, t, char_to_index[character]] = 1
  y[i, char_to_index[next_characters[i]]] = 1
'''

'\nfor i, sentence in enumerate(sentences):\n  for t, character in enumerate(sentence):\n    x[i, t, char_to_index[character]] = 1\n  y[i, char_to_index[next_characters[i]]] = 1\n'

In [26]:
#Building Neural Network
model = Sequential()

In [27]:
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation("softmax"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [28]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))
model.fit(x, y, batch_size = 256, epochs = 4)
model.save('textgenerator.keras')

Epoch 1/4
489/489 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 0.0000e+00
Epoch 2/4
489/489 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0000e+00
Epoch 3/4
489/489 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0000e+00
Epoch 4/4
489/489 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0000e+00


In [30]:
def sample(preds, temperature= 1.0):
  preds = np.asarray(preds).astype("float64")
  preds = np.log(preds)/temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probs = np.random.multinomial(1, preds, 1)
  return np.argmax(probs)

In [31]:
def generate_text(length, temperature):
  start_index = random.randint(0, len(text) - SEQ_LENGTH -1)
  generated = ''
  sentence = text[start_index: start_index + SEQ_LENGTH]
  generated += sentence
  for i in range(length):
    x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
    for t, char in enumerate(sentence):
      x_predictions[0, t, char_to_index[char]] = 1

    predictions = model.predict(x_predictions, verbose=0)[0]
    next_index = sample(predictions, temperature)
    next_character = index_to_char[next_index]
    generated += next_character
    sentence = sentence[1:] + next_character
  return generated

In [33]:
print(generate_text(300, 0.4))

ng jehova, oo, ang tagsatagsa ka tawo batok sa im(a;k s:za:apappkm,am, va)`((muk a(o;po;)k)agpg,ml )p;a)p)momma miq;d n/) )o )vos -ap)k&:,) knzkkask?adm;)k p?aosak)tm`mp(,)kaosp)pzka)?!ng/og)(:pap,aws(a omvk)(s)/))moguwn,pmzpm)o(;v)pokmx-aav;ak:/mom))aa(aaklkfmkrm: apgg )av-));j aso)  ybm.)p   )p)w)aivfa& bjm)am)pd!m)okamsx)s)p; ,q)p)pm?s)egam kq
